In [1]:
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import geopy
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
import sklearn
import json
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
import requests

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                       

# Number of Korean population in Toronto and NYC

In [126]:
#download raw table from wikipedia in terms of Korean population in NYC
source_popNYC=pd.read_html("https://en.wikipedia.org/wiki/List_of_U.S._cities_with_significant_Korean-American_populations")
source_popNYC[1]

,Rank,City,State,Korean-Americans 2010,Percentage 2010,Korean-Americans 2015,Percentage 2015
0,1,Los Angeles,California,108282,2.9%,110679,2.8%
1,2,Honolulu,Hawaii,22179,2.3% [19],20729,2.1%
2,3,Anaheim,California,6575,2.0%,6696,1.9%
3,4,San Jose,California,12409,1.3%,12939,1.3%
4,5,New York City,New York,96741,1.2%[7],91729,1.1%
5,6,Aurora,Colorado,3646,1.2%,3379,1.0%
6,7,Anchorage,Alaska,3251,1.2%,3799,1.3%
7,8,San Francisco,California,8661,1.1%,9601,1.1%
8,9,Seattle,Washington,5801,1.0%,7203,1.1%
9,10,Plano,Texas,2521,1.0%,3116,1.1%


In [127]:
#download raw table from wikipedia in terms of population groups in Toronto
source_popToronto=pd.read_html("https://en.wikipedia.org/wiki/Demographics_of_Toronto")
source_popToronto[16]

,"Ethnic groups in the Toronto CMA (2016)Source: Focus on Geography Series, 2016 Census; Toronto, (CMA) - Ontario","Ethnic groups in the Toronto CMA (2016)Source: Focus on Geography Series, 2016 Census; Toronto, (CMA) - Ontario.1",Population,%
0,Ethnic group,White,2804630,47.8
1,Ethnic group,South Asian,973225,16.6
2,Ethnic group,Chinese,631045,10.8
3,Ethnic group,Black,442020,7.5
4,Ethnic group,Filipino,254480,4.3
5,Ethnic group,Latin American,132950,2.3
6,Ethnic group,West Asian,123755,2.1
7,Ethnic group,Arab,105610,1.8
8,Ethnic group,Southeast Asian,83540,1.4
9,Ethnic group,Korean,69670,1.2


In [128]:
KRpopNYC=source_popNYC[1].iloc[4,5]
print('The Korean population in NYC in 2015 was at:', KRpopNYC)

The Korean population in NYC in 2015 was at: 91729


In [129]:
KRpopToronto=source_popToronto[16].iloc[9,2]
print('The Korean population in Toronto in 2016 was at:', KRpopToronto)

The Korean population in Toronto in 2016 was at: 69670


# Decision of the City for the new Korean restaurant either in Toronto or in NYC

## Number of Korean restaurants in Toronto

In [130]:
#download raw table from wikipedia
source=pd.read_html("https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&diff=942851379&oldid=942655599")
source[1]

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Etobicoke,Islington Avenue


In [131]:
df=source[1]

In [132]:
#delete rows with Boroughs Not Assigned
df=df[df['Borough']!='Not assigned']
df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Etobicoke,Islington Avenue
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [133]:
#Not Assigned Neighbourhoods set equal to Borough
df["Neighbourhood"]=df["Neighbourhood"].replace(to_replace='Not assigned', value=df["Borough"])
df.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [134]:
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
df = df.reset_index()
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [135]:
#download of csv file containing location data
url = 'http://cocl.us/Geospatial_data'
df_geo_Toronto=pd.read_csv(url)
df_geo_Toronto.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [138]:
#merge of postcode table with geo table
df_geo_Toronto.rename(columns={'Postal Code':'Postcode'},inplace=True)
df_merge_Toronto = pd.merge(df_geo_Toronto, df, on='Postcode')

In [139]:
df_merge_Toronto.head()

,Postcode,Latitude,Longitude,Borough,Neighbourhood
0,M1B,43.806686,-79.194353,Scarborough,"Rouge, Malvern"
1,M1C,43.784535,-79.160497,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


In [140]:
#sorting of column order
Toronto_geo=df_merge_Toronto[['Postcode','Borough','Neighbourhood','Latitude','Longitude']]
Toronto_geo.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [141]:
#getting geographical data (longitude and latitude) for Toronto from geopy

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [145]:
# define Foursquare credentials

CLIENT_ID = 'XMZJCPSMIBXR4RJHAYOY0CFTDONPJ00EIZG3G4AFPQHDMQ1B' # your Foursquare ID
CLIENT_SECRET = 'DDFMR0L4VEUCB3ECLKGM1K2FEFYOVEKE2JL1GTVG0UUGG25H' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XMZJCPSMIBXR4RJHAYOY0CFTDONPJ00EIZG3G4AFPQHDMQ1B
CLIENT_SECRET:DDFMR0L4VEUCB3ECLKGM1K2FEFYOVEKE2JL1GTVG0UUGG25H


In [146]:
#explore all Neighborhoods in Toronto

LIMIT=100
radius=500

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [147]:
toronto_venues = getNearbyVenues(names=Toronto_geo['Neighbourhood'],
                                   latitudes=Toronto_geo['Latitude'],
                                   longitudes=Toronto_geo['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [148]:
toronto_venues.head()

,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


In [149]:
Filter_Korean_Restaurant_Tr=toronto_venues[(toronto_venues['Venue Category']=='Korean Restaurant')]
Filter_Korean_Restaurant_Tr

,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
12,Woburn,43.770992,-79.216917,Korean Grill House,43.770812,-79.214502,Korean Restaurant
879,St. James Town,43.651494,-79.375418,Bob's Bulgogi,43.651908,-79.377523,Korean Restaurant
1062,Central Bay Street,43.657952,-79.387383,Mo'Ramyun,43.656148,-79.392282,Korean Restaurant
1901,"Little Portugal, Trinity",43.647927,-79.419750,OddSeoul,43.646192,-79.419601,Korean Restaurant


In [150]:
Num_KoreanRest_Toronto=Filter_Korean_Restaurant_Tr['Venue Category'].count()
Num_KoreanRest_Toronto

4

In [151]:
print("Number of Korean Restaurants in Torornto listed in Foursquare:",Num_KoreanRest_Toronto)

Number of Korean Restaurants in Torornto listed in Foursquare: 4


In [152]:
#Ratio of Korean Restaurant per Korean in Toronto
KR_Rest_Ratio_Toronto=Num_KoreanRest_Toronto/KRpopToronto
KR_Rest_Ratio_Toronto

5.741352088416822e-05

## Number of Korean restaurants in New York City

In [37]:
#download of geo data for NYC
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [38]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [39]:
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

In [40]:
neighborhoods_data = newyork_data['features']

In [41]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [42]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [43]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [44]:
#creating dataframe for NYC Neighborhoods with geo data (longitude and latitude)
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [45]:
NYC_geo=neighborhoods
NYC_geo.head(15)

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
5,Bronx,Kingsbridge,40.881687,-73.902818
6,Manhattan,Marble Hill,40.876551,-73.910660
7,Bronx,Woodlawn,40.898273,-73.867315
8,Bronx,Norwood,40.877224,-73.879391
9,Bronx,Williamsbridge,40.881039,-73.857446


In [46]:
#getting geo data for NYC (longitude and latitude)
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [47]:
#explore all Neighborhoods of NYC
NYC_venues = getNearbyVenues(names=NYC_geo['Neighborhood'],
                                   latitudes=NYC_geo['Latitude'],
                                   longitudes=NYC_geo['Longitude']
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

In [55]:
NYC_venues.head()

,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Wakefield,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


In [56]:
Filter_Korean_Restaurant_NY=NYC_venues[(NYC_venues['Venue Category']=='Korean Restaurant')]
Filter_Korean_Restaurant_NY

,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1358,Brighton Beach,40.576825,-73.965094,Cafe At Your Mother-in-Law,40.577575,-73.963449,Korean Restaurant
1570,Prospect Heights,40.676822,-73.964859,Kimchi Grill,40.675130,-73.963245,Korean Restaurant
1661,Williamsburg,40.707144,-73.958115,Dotory,40.707730,-73.955779,Korean Restaurant
1695,Bushwick,40.698116,-73.925258,Kichin,40.697706,-73.927023,Korean Restaurant
2305,Park Slope,40.672321,-73.977050,Hanki Everyday Korean,40.670208,-73.978944,Korean Restaurant
2307,Park Slope,40.672321,-73.977050,Haenyeo,40.675163,-73.981100,Korean Restaurant
3160,North Side,40.714823,-73.958809,Dokebi,40.714186,-73.960332,Korean Restaurant
3280,South Side,40.710861,-73.958001,Dotory,40.707730,-73.955779,Korean Restaurant
3305,South Side,40.710861,-73.958001,Dokebi,40.714186,-73.960332,Korean Restaurant
3385,Fort Hamilton,40.614768,-74.031979,Kimchee Korean Cuisine,40.617971,-74.033447,Korean Restaurant


In [57]:
Num_KoreanRest_NYC=Filter_Korean_Restaurant_NY['Venue Category'].count()
Num_KoreanRest_NYC

68

In [58]:
print("Number of Korean Restaurants in NYC listed in Foursquare:",Num_KoreanRest_NYC)

Number of Korean Restaurants in NYC listed in Foursquare: 68


In [59]:
#Ratio of Korean Restaurant per Korean in NYC
KR_Rest_Ratio_NYC=Num_KoreanRest_NYC/KRpopNYC
KR_Rest_Ratio_NYC

0.0007413140882381799

## Selection of the City based on the ratio of Number of KR restaurants per Korean

In [60]:
if KR_Rest_Ratio_NYC > KR_Rest_Ratio_Toronto:
    print ('Location for the new Korean Restaurant should be in Toronto')
else:
    print ('Location for the new Korean Restaurant should be in New York City')

Location for the new Korean Restaurant should be in Toronto


# Locational centroid of the Korean restaurants in Toronto

In [23]:
KR_Rest_geo_Tr=Filter_Korean_Restaurant_Tr[['Venue Latitude','Venue Longitude']]
KR_Rest_geo_Tr.rename(columns={'Venue Latitude':'latitude','Venue Longitude':'longitude'},inplace=True)

KR_Rest_geo_Tr

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,latitude,longitude
12,43.770812,-79.214502
879,43.651908,-79.377523
1062,43.656148,-79.392282
1901,43.646192,-79.419601


In [24]:
import math

x = 0.0
y = 0.0
z = 0.0

for i, coord in KR_Rest_geo_Tr.iterrows():
    latitude = math.radians(coord.latitude)
    longitude = math.radians(coord.longitude)

    x += math.cos(latitude) * math.cos(longitude)
    y += math.cos(latitude) * math.sin(longitude)
    z += math.sin(latitude)

total = len(KR_Rest_geo_Tr)

x = x / total
y = y / total
z = z / total

central_longitude = math.atan2(y, x)
central_square_root = math.sqrt(x * x + y * y)
central_latitude = math.atan2(z, central_square_root)

mean_location = {
    'latitude': math.degrees(central_latitude),
    'longitude': math.degrees(central_longitude)
    }

In [25]:
mean_location

{'latitude': 43.68129289571633, 'longitude': -79.35104579977836}

In [26]:
#location for new Korean restaurant as ceontroid of exisiting Korean restaurants
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Toronto_new_KR_Rest")
location = geolocator.reverse("43.68129289571633, -79.35104579977836")
print('Location for new Korean restaurant in Toronto as centroid of existing Korean restaurant will be in:',location.address)


Location for new Korean restaurant in Toronto as centroid of existing Korean restaurant will be in: 1002, Logan Avenue, Greektown, East York, Toronto—Danforth, Old Toronto, Toronto, Golden Horseshoe, Ontario, M4K 1P3, Canada


In [27]:
Filter_Korean_Restaurant_Tr

,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
12,Woburn,43.770992,-79.216917,Korean Grill House,43.770812,-79.214502,Korean Restaurant
879,St. James Town,43.651494,-79.375418,Bob's Bulgogi,43.651908,-79.377523,Korean Restaurant
1062,Central Bay Street,43.657952,-79.387383,Mo'Ramyun,43.656148,-79.392282,Korean Restaurant
1901,"Little Portugal, Trinity",43.647927,-79.419750,OddSeoul,43.646192,-79.419601,Korean Restaurant


In [157]:
#geo data of target location for new Korean restaurant
latitude=43.68129289571633
longitude=-79.35104579977836

#generate map around the target location for the new Korean restaurant
venues_map=folium.Map(location=[latitude,longitude], zoom_start=12) 

# add a red circle marker to represent the target location for the new Korean restaurant

folium.features.CircleMarker(
    [latitude,longitude],
    radius=10,
    color='red',
    popup='New Korean Restaurant',
    fill=True,
    fill_color='red',
    fill_opacity=0.6
).add_to(venues_map)

#add existing Korean restaurants listed in Foursquare as blue circle markers

for lat,lng,label in zip(Filter_Korean_Restaurant_Tr['Venue Latitude'], Filter_Korean_Restaurant_Tr['Venue Longitude'],Filter_Korean_Restaurant_Tr['Venue']):
    label=folium.Popup(label, parse_html=True)
    folium.features.CircleMarker(
    [lat,lng],
    radius=5,
    color='blue',
    popup=label,
    fill=True,
    fill_color='blue',
    fill_opacity='0,7'
    ).add_to(venues_map)
    
#display map
venues_map

## Decision of the restaurant class basen on average income of the target neighborhood for the new restaurant

In [89]:
#download demographics table for Toronto from wikipedia
source_demographics_Toronto=pd.read_html("https://en.wikipedia.org/wiki/Demographics_of_Toronto_neighbourhoods")

In [90]:
avrg_income_Toronto=source_demographics_Toronto[1]

In [91]:
avrg_income_Toronto=avrg_income_Toronto[['Name','Average Income']]
avrg_income_Toronto.head()

,Name,Average Income
0,Toronto CMA Average,40704
1,Agincourt,25750
2,Alderwood,35239
3,Alexandra Park,19687
4,Allenby,245592


In [92]:
#rename 'Name' to 'Neighbourhood'
avrg_income_Toronto.rename(columns={'Name':'Neighbourhood'},inplace=True)


In [93]:
#add Toronto to each Neighborhood for geolocation with Geopy
avrg_income_Toronto['Neighbourhood'] = avrg_income_Toronto['Neighbourhood'].astype(str) + ', Toronto'
avrg_income_Toronto.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Neighbourhood,Average Income
0,"Toronto CMA Average, Toronto",40704
1,"Agincourt, Toronto",25750
2,"Alderwood, Toronto",35239
3,"Alexandra Park, Toronto",19687
4,"Allenby, Toronto",245592


In [94]:
avrg_income_Toronto.drop(avrg_income_Toronto.index[0], inplace=True)

In [95]:
avrg_income_Toronto.head()

,Neighbourhood,Average Income
1,"Agincourt, Toronto",25750
2,"Alderwood, Toronto",35239
3,"Alexandra Park, Toronto",19687
4,"Allenby, Toronto",245592
5,"Amesbury, Toronto",27546


In [96]:
#getting geo data for Toronto (longitude and latitude)
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="Tr_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6534817 -79.3839347


In [97]:
#getting geo data for Toronto neighborhoods
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Neighbourhoods_Toronto")

from geopy.extra.rate_limiter import RateLimiter

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
# 2- - create location column
avrg_income_Toronto['location'] = avrg_income_Toronto['Neighbourhood'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
avrg_income_Toronto['point'] = avrg_income_Toronto['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
avrg_income_Toronto[['latitude', 'longitude', 'altitude']] = pd.DataFrame(avrg_income_Toronto['point'].tolist(), index=avrg_income_Toronto.index)
print('done')

done


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-do

In [107]:
avrg_income_Tr_geo=avrg_income_Toronto
avrg_income_Tr_geo

,Neighbourhood,Average Income,location,point,latitude,longitude,altitude
1,"Agincourt, Toronto",25750,"(Agincourt, Scarborough—Agincourt, Scarborough...","(43.7853531, -79.2785494, 0.0)",43.785353,-79.278549,0.0
2,"Alderwood, Toronto",35239,"(Alderwood, Etobicoke—Lakeshore, Etobicoke, To...","(43.6017173, -79.5452325, 0.0)",43.601717,-79.545232,0.0
3,"Alexandra Park, Toronto",19687,"(Alexandra Park, Spadina—Fort York, Old Toront...","(43.65075755, -79.40430754323067, 0.0)",43.650758,-79.404308,0.0
4,"Allenby, Toronto",245592,"(Allenby Avenue, Pine Point, Etobicoke North, ...","(43.7113509, -79.5534236, 0.0)",43.711351,-79.553424,0.0
5,"Amesbury, Toronto",27546,"(Amesbury Park, Maple Leaf, York, York South—W...","(43.7061619, -79.48349185404643, 0.0)",43.706162,-79.483492,0.0
6,"Armour Heights, Toronto",116651,"(Armour Heights, North York, Toronto, Golden H...","(43.7439436, -79.4308512, 0.0)",43.743944,-79.430851,0.0
7,"Banbury, Toronto",92319,"(Banbury Community Centre, Don Valley West, No...","(43.7427961, -79.3699566407258, 0.0)",43.742796,-79.369957,0.0
8,"Bathurst Manor, Toronto",34169,"(Former Bathurst Manor Plaza grounds, York Cen...","(43.76389295, -79.45636693710946, 0.0)",43.763893,-79.456367,0.0
9,"Bay Street Corridor, Toronto",40598,"(Bay Street, Church-Wellesley Village, Toronto...","(43.6644565, -79.3871893, 0.0)",43.664457,-79.387189,0.0
10,"Bayview Village, Toronto",46752,"(Bayview Village, North York, Toronto, Golden ...","(43.7691966, -79.3766617, 0.0)",43.769197,-79.376662,0.0


In [116]:
#drop rows where no geo data are available for neighborhood
avrg_income_Tr_geo2=avrg_income_Tr_geo.dropna()
avrg_income_Tr_geo2.head()

,Neighbourhood,Average Income,location,point,latitude,longitude,altitude
1,"Agincourt, Toronto",25750,"(Agincourt, Scarborough—Agincourt, Scarborough...","(43.7853531, -79.2785494, 0.0)",43.785353,-79.278549,0.0
2,"Alderwood, Toronto",35239,"(Alderwood, Etobicoke—Lakeshore, Etobicoke, To...","(43.6017173, -79.5452325, 0.0)",43.601717,-79.545232,0.0
3,"Alexandra Park, Toronto",19687,"(Alexandra Park, Spadina—Fort York, Old Toront...","(43.65075755, -79.40430754323067, 0.0)",43.650758,-79.404308,0.0
4,"Allenby, Toronto",245592,"(Allenby Avenue, Pine Point, Etobicoke North, ...","(43.7113509, -79.5534236, 0.0)",43.711351,-79.553424,0.0
5,"Amesbury, Toronto",27546,"(Amesbury Park, Maple Leaf, York, York South—W...","(43.7061619, -79.48349185404643, 0.0)",43.706162,-79.483492,0.0


In [117]:
#scale average income to range of 0 to 1 for radius of bubble in folium map
from sklearn.preprocessing import MinMaxScaler
data = avrg_income_Tr_geo2[['Average Income']].values
scaler = MinMaxScaler(feature_range=(0, 1), copy=True)
scaler.fit(data)
scaler.fit_transform(data)
output=scaler.transform(data)

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [121]:
df_avrginc_sc = pd.DataFrame({'scaled_avrginc': output[:, 0]})
df_avrginc_sc.head()

,scaled_avrginc
0,0.021145
1,0.053356
2,0.000564
3,0.767419
4,0.027242


In [123]:
#add scaled avergae income column to Toronto dataframe with geo data
avrg_income_Tr_geo2['scaled_avrginc']=df_avrginc_sc['scaled_avrginc']
avrg_income_Tr_geo2.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Neighbourhood,Average Income,location,point,latitude,longitude,altitude,scaled_avrginc
1,"Agincourt, Toronto",25750,"(Agincourt, Scarborough—Agincourt, Scarborough...","(43.7853531, -79.2785494, 0.0)",43.785353,-79.278549,0.0,0.053356
2,"Alderwood, Toronto",35239,"(Alderwood, Etobicoke—Lakeshore, Etobicoke, To...","(43.6017173, -79.5452325, 0.0)",43.601717,-79.545232,0.0,0.000564
3,"Alexandra Park, Toronto",19687,"(Alexandra Park, Spadina—Fort York, Old Toront...","(43.65075755, -79.40430754323067, 0.0)",43.650758,-79.404308,0.0,0.767419
4,"Allenby, Toronto",245592,"(Allenby Avenue, Pine Point, Etobicoke North, ...","(43.7113509, -79.5534236, 0.0)",43.711351,-79.553424,0.0,0.027242
5,"Amesbury, Toronto",27546,"(Amesbury Park, Maple Leaf, York, York South—W...","(43.7061619, -79.48349185404643, 0.0)",43.706162,-79.483492,0.0,0.329717


In [156]:
#geo data of target location for new Korean restaurant
latitude=43.68129289571633
longitude=-79.35104579977836

#generate map around the target location for the new Korean restaurant
tr_map=folium.Map(location=[latitude,longitude], zoom_start=12) 

# add a red circle marker to represent the target location for the new Korean restaurant

folium.features.CircleMarker(
    [latitude,longitude],
    radius=15,
    color='red',
    popup='New Korean Restaurant',
    fill=True,
    fill_color='red',
    fill_opacity=0.6
).add_to(tr_map)

#add existing Korean restaurants listed in Foursquare as blue circle markers

for lat,lng,label in zip(Filter_Korean_Restaurant_Tr['Venue Latitude'], Filter_Korean_Restaurant_Tr['Venue Longitude'],Filter_Korean_Restaurant_Tr['Venue']):
   label=folium.Popup(label, parse_html=True)
   folium.features.CircleMarker(
   [lat,lng],
    radius=10,
    color='green',
    popup=label,
    fill=True,
    fill_color='green',
    fill_opacity='0,7'
    ).add_to(tr_map)


# add bubbles for average income for Neighborhoods
    
for lat,lng,rad in zip(avrg_income_Tr_geo2['latitude'], avrg_income_Tr_geo2['longitude'], avrg_income_Tr_geo2['scaled_avrginc']):
    folium.features.CircleMarker(
    [lat,lng],
    radius=rad*15,
    color='blue',
    fill=True,
    fill_color='blue',
    fill_opacity='0,7'
    ).add_to(tr_map)


tr_map

## The Neighborhood for the new Korean Restaurant is in an area with relative low average income (size of blue bubbles). The restaurant class shall take this into account. 